In [1]:
import h5py
import glob
from ml4cvd.tensor_from_file import _mri_tensor_4d, _mri_hd5_to_structured_grids
from ml4cvd.defines import MRI_LAX_4CH_SEGMENTED_CHANNEL_MAP, MRI_LAX_3CH_SEGMENTED_CHANNEL_MAP, MRI_FRAMES
from ml4cvd.tensor_from_file import _mri_project_grids
from vtk.util import numpy_support as ns
import vtk
import numpy as np
import pandas as pd

In [ ]:
hd5s = glob.glob('/mnt/disks/sax-lax-40k-lvm/2020-01-29/*.hd5')
results = {f'dice_{d}': [] for d in range(MRI_FRAMES)}
results['patientid'] = []

for i, hd5 in enumerate(hd5s):
    with h5py.File(hd5, 'r') as ff:
        if 'ukb_cardiac_mri' not in ff:
            continue
        results['patientid'].append(hd5.split('/')[-1].replace('.hd5', ''))
            
        ds_3ch = _mri_hd5_to_structured_grids(ff, 'cine_segmented_lax_3ch_annotated_', view_name='cine_segmented_lax_3ch', concatenate=True, save_path=None, order='F')
        ds_4ch = _mri_hd5_to_structured_grids(ff, 'cine_segmented_lax_4ch_annotated_', view_name='cine_segmented_lax_4ch', concatenate=True, save_path=None, order='F')
        ds_4ch_3ch = _mri_project_grids(ds_3ch, ds_4ch, 'cine_segmented_lax_3ch_annotated_')
        ds_3ch_4ch = _mri_project_grids(ds_4ch, ds_3ch, 'cine_segmented_lax_4ch_annotated_')
        for t in range(MRI_FRAMES):
            arr_4onto3 = ns.vtk_to_numpy(ds_3ch_4ch[0].GetCellData().GetArray(f'cine_segmented_lax_4ch_annotated__projected_{t}'))
            arr_3 = ns.vtk_to_numpy(ds_3ch_4ch[0].GetCellData().GetArray(f'cine_segmented_lax_3ch_annotated__{t}'))
            arr_3onto4 = ns.vtk_to_numpy(ds_4ch_3ch[0].GetCellData().GetArray(f'cine_segmented_lax_3ch_annotated__projected_{t}'))
            arr_4 = ns.vtk_to_numpy(ds_4ch_3ch[0].GetCellData().GetArray(f'cine_segmented_lax_4ch_annotated__{t}'))
            cond1 = arr_4onto3 > 0
            inters1 = np.sum(((arr_4onto3==MRI_LAX_4CH_SEGMENTED_CHANNEL_MAP['LA_cavity']) & cond1) & \
                             ((arr_3==MRI_LAX_3CH_SEGMENTED_CHANNEL_MAP['left_atrium']) & cond1))
            union1 = np.sum(((arr_4onto3==MRI_LAX_4CH_SEGMENTED_CHANNEL_MAP['LA_cavity']) & cond1) | \
                            ((arr_3==MRI_LAX_3CH_SEGMENTED_CHANNEL_MAP['left_atrium']) & cond1))
            cond2 = arr_3onto4 > 0
            inters2 = np.sum(((arr_3onto4==MRI_LAX_3CH_SEGMENTED_CHANNEL_MAP['left_atrium']) & cond2) & \
                             ((arr_4==MRI_LAX_4CH_SEGMENTED_CHANNEL_MAP['LA_cavity']) & cond2))
            union2 = np.sum(((arr_3onto4==MRI_LAX_3CH_SEGMENTED_CHANNEL_MAP['left_atrium']) & cond2) | \
                            ((arr_4==MRI_LAX_4CH_SEGMENTED_CHANNEL_MAP['LA_cavity']) & cond2))
            results[f'dice_{t}'].append(np.mean([inters1/union1, inters2/union2]))

/usr/local/lib/python3.6/dist-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in long_scalars


In [ ]:
results_df = pd.DataFrame(results)
results_df

In [ ]:
arr_annot_3ch = _mri_tensor_4d(ff, 'cine_segmented_lax_3ch_annotated_', concatenate=True)
#arr_3ch = _mri_tensor_4d(ff, 'cine_segmented_lax_3ch')
arr_annot_4ch = _mri_tensor_4d(ff, 'cine_segmented_lax_4ch_annotated_', concatenate=True)
#arr_4ch = _mri_tensor_4d(ff, 'cine_segmented_lax_4ch')

In [ ]:
ds_3ch = _mri_hd5_to_structured_grids(ff, 'cine_segmented_lax_3ch_annotated_', view_name='cine_segmented_lax_3ch', concatenate=True, save_path=None, order='F')
ds_4ch = _mri_hd5_to_structured_grids(ff, 'cine_segmented_lax_4ch_annotated_', view_name='cine_segmented_lax_4ch', concatenate=True, save_path=None, order='F')

In [ ]:
from ml4cvd.tensor_from_file import _mri_project_grids
ds_4ch_3ch = _mri_project_grids(ds_3ch, ds_4ch, 'cine_segmented_lax_3ch_annotated_')
ds_3ch_4ch = _mri_project_grids(ds_4ch, ds_3ch, 'cine_segmented_lax_4ch_annotated_')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
f, ax = plt.subplots(1, 2)
ax[0].imshow(arr_annot_3ch[:, :, 0, 0]==2)
ax[1].imshow(arr_annot_4ch[:, :, 0, 0]==10)

In [ ]:
import vtk
from vtk.util import numpy_support as ns
writer = vtk.vtkXMLStructuredGridWriter()
writer.SetInputData(ds_4ch_3ch[0])
writer.SetFileName('3ch_to_4ch.vts')
writer.Update()
writer.SetInputData(ds_3ch_4ch[0])
writer.SetFileName('4ch_to_3ch.vts')
writer.Update()

In [ ]:
writer.SetInputData(ds_3ch[0])
writer.SetFileName('3ch.vts')
writer.Update()

In [ ]:
import numpy as np
for t in range(MRI_FRAMES):
    arr_4onto3 = ns.vtk_to_numpy(ds_3ch_4ch[0].GetCellData().GetArray(f'cine_segmented_lax_4ch_annotated__projected_{t}'))
    arr_3 = ns.vtk_to_numpy(ds_3ch_4ch[0].GetCellData().GetArray(f'cine_segmented_lax_3ch_annotated__{t}'))
    arr_3onto4 = ns.vtk_to_numpy(ds_4ch_3ch[0].GetCellData().GetArray(f'cine_segmented_lax_3ch_annotated__projected_{t}'))
    arr_4 = ns.vtk_to_numpy(ds_4ch_3ch[0].GetCellData().GetArray(f'cine_segmented_lax_4ch_annotated__{t}'))
    cond1 = arr_4onto3 > 0
    inters1 = np.sum(((arr_4onto3==MRI_LAX_4CH_SEGMENTED_CHANNEL_MAP['LA_cavity']) & cond1) & \
                     ((arr_3==MRI_LAX_3CH_SEGMENTED_CHANNEL_MAP['left_atrium']) & cond1))
    union1 = np.sum(((arr_4onto3==MRI_LAX_4CH_SEGMENTED_CHANNEL_MAP['LA_cavity']) & cond1) | \
                    ((arr_3==MRI_LAX_3CH_SEGMENTED_CHANNEL_MAP['left_atrium']) & cond1))
    cond2 = arr_3onto4 > 0
    inters2 = np.sum(((arr_3onto4==MRI_LAX_3CH_SEGMENTED_CHANNEL_MAP['left_atrium']) & cond2) & \
                     ((arr_4==MRI_LAX_4CH_SEGMENTED_CHANNEL_MAP['LA_cavity']) & cond2))
    union2 = np.sum(((arr_3onto4==MRI_LAX_3CH_SEGMENTED_CHANNEL_MAP['left_atrium']) & cond2) | \
                    ((arr_4==MRI_LAX_4CH_SEGMENTED_CHANNEL_MAP['LA_cavity']) & cond2))
    print(inters1/union1, inters2/union2)

In [ ]:
import numpy as np
cond1 = arr_4onto3 > 0
inters1 = np.sum(((arr_4onto3==MRI_LAX_4CH_SEGMENTED_CHANNEL_MAP['LA_cavity']) & cond1) & \
                 ((arr_3==MRI_LAX_3CH_SEGMENTED_CHANNEL_MAP['left_atrium']) & cond1))
union1 = np.sum(((arr_4onto3==MRI_LAX_4CH_SEGMENTED_CHANNEL_MAP['LA_cavity']) & cond1) | \
                ((arr_3==MRI_LAX_3CH_SEGMENTED_CHANNEL_MAP['left_atrium']) & cond1))

In [ ]:
cond2 = arr_3onto4 > 0
inters2 = np.sum(((arr_3onto4==MRI_LAX_3CH_SEGMENTED_CHANNEL_MAP['left_atrium']) & cond2) & \
                 ((arr_4==MRI_LAX_4CH_SEGMENTED_CHANNEL_MAP['LA_cavity']) & cond2))
union2 = np.sum(((arr_3onto4==MRI_LAX_3CH_SEGMENTED_CHANNEL_MAP['left_atrium']) & cond2) | \
                ((arr_4==MRI_LAX_4CH_SEGMENTED_CHANNEL_MAP['LA_cavity']) & cond2))

In [ ]:
print(inters1/union1, inters2/union2)

In [ ]:
arr_3onto3 = ns.vtk_to_numpy(ds_3ch_4ch[0].GetCellData().GetArray('cine_segmented_lax_4ch_annotated__projected_0'))
arr_3 = ns.vtk_to_numpy(ds_3ch_4ch[0].GetCellData().GetArray('cine_segmented_lax_3ch_annotated__0'))
arr_3onto4 = ns.vtk_to_numpy(ds_4ch_3ch[0].GetCellData().GetArray('cine_segmented_lax_3ch_annotated__projected_0'))
arr_4 = ns.vtk_to_numpy(ds_4ch_3ch[0].GetCellData().GetArray('cine_segmented_lax_4ch_annotated__0'))